In [1]:
!pip install transformers gradio
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import os
from huggingface_hub import login

In [3]:
# 替換為你的Hugging Face API Token
my_read_token = "請改成你的read token"
os.environ["HUGGINGFACE_TOKEN"] = my_read_token
# 登錄Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
model_name = "taide/TAIDE-LX-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# 加载 tokenizer 和 模型

peft_name = "mark1098/TAIDE-LX-7B-Chat-Medical-Fintune"
model = PeftModel.from_pretrained(model, peft_name)
model = model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/813k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

In [5]:
import gradio as gr
import re
# 定义一个生成文本的函数
def chat_with_model(input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    # outputs = model.generate(inputs.input_ids, max_new_tokens=150, pad_token_id=tokenizer.pad_token_id)
    # prompt = "每天肚子痛是啥狀況？"
    messages = [
        {"role": "system", "content": "你是一位專業的醫療人員，請用心且專業的以三到五句話回答問題。"},
        {"role": "user", "content": input_text}
    ]

    # 將消息合併成一個文本輸入
    def format_messages(messages):
        formatted_messages = ""
        for message in messages:
            if message['role'] == 'system':
                formatted_messages += f"[SYSTEM] {message['content']}\n"
            elif message['role'] == 'user':
                formatted_messages += f"[USER] {message['content']}\n"
        return formatted_messages

    # 格式化消息
    formatted_text = format_messages(messages)
    print("Formatted text:", formatted_text)

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    print("Generated text with chat template:", text)


    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

    # 印出model_inputs進行檢查
    print("Model inputs:", model_inputs)

    # 產生文本
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=90,
        #eos_token_id=tokenizer.encode('<|eot_id|>')[0],
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=model_inputs.attention_mask,
    repetition_penalty=1.6,  # 增加 repetition_penalty
        top_k=50,  # 設定最高k個概率詞
        # stop_token=tokenizer.eos_token,  # 確保模型在遇到停止標記時中斷
        do_sample=True,
        top_p=0.15,  # 新增 top_p 參數
        temperature=0.15,  # 設定 temperature 參數
        #forced_eos_token_id=tokenizer.encode('</s>')[0]
    )
    generated_ids = generated_ids[:, model_inputs.input_ids.shape[-1]:]


    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # 清理生成的文本

    response = re.sub(r'</s>.*', '', response, flags=re.DOTALL)
    response = re.sub(r'</s>.*', '</s>', response)
    response = re.sub(r'\[.*?\]', '', response)  # 去除方括號內的內容
    response = re.sub(r'</?[^>]+>', '', response)  # 去除HTML標籤
    response = re.sub(r'dress|dressing', '', response, flags=re.IGNORECASE)  # 去除 "dress" 和 "dressing"
    response = re.sub(r'<<.*?>>', '', response)  # 去除 `<<SYS>>` 標籤
    response = re.sub(r'\[.*?\]', '', response)  # 去除 `[/EMBB]]` 標籤
    response = response.strip()  # 去除前後多餘的空格
    # 從最後開始查找句號並去除句號之後的語句
    def remove_after_last_period(text):
        last_period_index = max(text.rfind('。'), text.rfind('!'))
        if last_period_index != -1:
            return text[:last_period_index + 1]
        return text

    # 清理生成的文本
    response = remove_after_last_period(response).strip()
    return response

# 创建 Gradio 界面
iface = gr.Interface(fn=chat_with_model, inputs="text", outputs="text", title="Medical Chatbot", description="使用TAIDE-LX-7B-Chat-Medical-Fintune模型的医疗聊天机器人")

# 启动接口
iface.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eb01235af596422872.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
model = model.merge_and_unload()
save_directory = "./full_model_with_peft"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./full_model_with_peft/tokenizer_config.json',
 './full_model_with_peft/special_tokens_map.json',
 './full_model_with_peft/tokenizer.model',
 './full_model_with_peft/added_tokens.json',
 './full_model_with_peft/tokenizer.json')

In [12]:
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make

Cloning into 'llama.cpp'...
remote: Enumerating objects: 32066, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 32066 (delta 57), reused 105 (delta 50), pack-reused 31949 (from 1)
Receiving objects: 100% (32066/32066), 55.52 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (22967/22967), done.
/content/llama.cpp
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wex

In [17]:

!python convert_hf_to_gguf.py /content/full_model_with_peft \
  --outfile /content/TAIDE-LX-7B-Chat-Medical-Fintune.gguf \
  --outtype f16

INFO:hf-to-gguf:Loading model: full_model_with_peft
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00006.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> F16, shape = {4096, 56064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float32 --> F16, shape = {11008, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> F16, shape = {4096, 4096}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float32 --> F16, sh

In [27]:
!./llama-quantize /content/TAIDE-LX-7B-Chat-Medical-Fintune.gguf /content/TAIDE-LX-7B-Chat-Medical-Fintune-q5_k_m.gguf q5_k_m

main: build = 3579 (fc4ca27b)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/TAIDE-LX-7B-Chat-Medical-Fintune.gguf' to '/content/TAIDE-LX-7B-Chat-Medical-Fintune-q5_k_m.gguf' as Q5_K_M
llama_model_loader: loaded meta data with 32 key-value pairs and 291 tensors from /content/TAIDE-LX-7B-Chat-Medical-Fintune.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = TAIDE LX 7B Chat
llama_model_loader: - kv   3:                       general.organization str              = Taide
llama_model_loader: - kv   4:                           general.finetune str              = Chat
ll

In [30]:
from huggingface_hub import HfApi
import os

api = HfApi()
HF_ACCESS_TOKEN = "改成你的write token"
model_id = "mark1098/TAIDE-LX-7B-Chat-Medical-Fintune.gguf"

api.create_repo(
    model_id,
    exist_ok=True,
    repo_type="model", # 上傳格式為模型
    use_auth_token=HF_ACCESS_TOKEN,
)
# upload the model to the hub
# upload model name includes the TAIDE-LX-7B-Chat-Medical-Fintune.gguf in same folder
for file in os.listdir():
    if file.endswith(".gguf"):
        model_name = file.lower()
        api.upload_file(
            repo_id=model_id,
            path_in_repo=model_name,
            path_or_fileobj=f"{os.getcwd()}/{file}",
            repo_type="model", # 上傳格式為模型
            use_auth_token=HF_ACCESS_TOKEN)

TAIDE-LX-7B-Chat-Medical-Fintune.gguf:   0%|          | 0.00/13.9G [00:00<?, ?B/s]